# Context Engine : MAS & MCP

Copyright 2025, Denis Rothman


**Building the Context Engine**

*From Prototype to pre-production: Hardening the Context Engine*


This notebook implements the evolution of a "Context Engine" from a prototype to a robust, pre-production multi-agent AI system. It showcases a sophisticated architecture featuring a central Planner that dynamically creates execution strategies based on high-level goals. The system deploys specialized agents—a Librarian for stylistic blueprints, a Researcher for factual retrieval, and a Writer for content generation.

A core principle demonstrated here is **dependency injection**, where all configurations, clients, and model parameters are explicitly passed, eliminating reliance on global variables and enhancing modularity. The Planner is hardened to produce structured JSON output, ensuring reliable parsing and execution. The agents collaborate through **context chaining**, allowing the output of one step to seamlessly become the input for the next.

This engine leverages a Retrieval-Augmented Generation (RAG) pipeline, using Pinecone for vector search across distinct knowledge and context namespaces. Communication is standardized via a custom Model Context Protocol (MCP). The final execution cell provides a practical example, tasking the engine with generating a creative story about the Apollo 11 moon landing, showcasing the synergy of the entire hardened system.


# GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/utils.py")
download_private_github_file("commons/agents.py")

✅ Successfully downloaded 'commons/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/agents.py' to './agents.py'


# 1.Installation and Setup 2.Initialize Clients

In [ ]:
# 1. Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


# 3.Helper Functions (LLM, Embeddings, and MCP)

In [ ]:
# === Imports for this section ===
import json
import time
import logging
import tiktoken
import re
import copy
from tenacity import retry, stop_after_attempt, wait_random_exponential
from openai import APIError # Import specific error for better handling

# === Configure Production-Level Logging ===
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# === LLM Interaction (Hardened with Dependency Injection) ===
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def call_llm_robust(system_prompt, user_prompt, client, generation_model, json_mode=False):
    """
    A centralized function to handle all LLM interactions with retries.
    UPGRADE: Now requires the 'client' and 'generation_model' objects to be passed in.
    """
    logging.info("Attempting to call LLM...")
    try:
        response_format = {"type": "json_object"} if json_mode else {"type": "text"}
        # UPGRADE: Uses the passed-in client and model name for the API call.
        response = client.chat.completions.create(
            model=generation_model,
            response_format=response_format,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        logging.info("LLM call successful.")
        return response.choices[0].message.content.strip()
    except APIError as e:
        logging.error(f"OpenAI API Error in call_llm_robust: {e}")
        raise e
    except Exception as e:
        logging.error(f"An unexpected error occurred in call_llm_robust: {e}")
        raise e

# === Embeddings (Hardened with Dependency Injection) ===
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text, client, embedding_model):
    """
    Generates embeddings for a single text query with retries.
    UPGRADE: Now requires the 'client' and 'embedding_model' objects.
    """
    text = text.replace("\n", " ")
    try:
        # UPGRADE: Uses the passed-in client and model name.
        response = client.embeddings.create(input=[text], model=embedding_model)
        return response.data[0].embedding
    except APIError as e:
        logging.error(f"OpenAI API Error in get_embedding: {e}")
        raise e
    except Exception as e:
        logging.error(f"An unexpected error occurred in get_embedding: {e}")
        raise e

# === Model Context Protocol (MCP) ===
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "2.0 (Context Engine)",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

# === Pinecone Interaction (Hardened with Dependency Injection) ===
def query_pinecone(query_text, namespace, top_k, index, client, embedding_model):
    """
    Embeds the query text and searches the specified Pinecone namespace.
    UPGRADE: Now requires 'index', 'client', and 'embedding_model' objects.
    """
    logging.info(f"Querying Pinecone namespace '{namespace}'...")
    try:
        # UPGRADE: Passes the necessary dependencies down to get_embedding.
        query_embedding = get_embedding(query_text, client=client, embedding_model=embedding_model)
        # UPGRADE: Uses the passed-in index object for the query.
        response = index.query(
            vector=query_embedding,
            namespace=namespace,
            top_k=top_k,
            include_metadata=True
        )
        logging.info("Pinecone query successful.")
        return response['matches']
    except Exception as e:
        logging.error(f"Error querying Pinecone (Namespace: {namespace}): {e}")
        raise e

# === Context Management Utility (New) ===
def count_tokens(text, model="gpt-4"):
    """Counts the number of tokens in a text string for a given model."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        # Fallback for models that might not be in the tiktoken registry
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))


logging.info("✅ Helper functions defined and upgraded.")

# 4.The Specialist Agents (Upgraded for Production)

In [ ]:
#4.The Specialist Agents (Fully Upgraded with Full Dependency Injection)
# -------------------------------------------------------------------------
# We now complete the upgrade of our specialist agents.
# The final step is to pass the configuration variables (like model names
# AND namespaces) as arguments, making the agents fully self-contained and
# removing all reliance on global variables.
# -------------------------------------------------------------------------

# === 4.1. Context Librarian Agent (Upgraded) ===
# *** Added 'namespace_context' argument ***
def agent_context_librarian(mcp_message, client, index, embedding_model, namespace_context):
    """
    Retrieves the appropriate Semantic Blueprint from the Context Library.
    UPGRADE: Now also accepts embedding_model and namespace configuration.
    """
    logging.info("[Librarian] Activated. Analyzing intent...")
    try:
        requested_intent = mcp_message['content'].get('intent_query')

        if not requested_intent:
            raise ValueError("Librarian requires 'intent_query' in the input content.")

        # UPGRADE: Pass all necessary dependencies to the hardened helper function.
        results = query_pinecone(
            query_text=requested_intent,
            # *** Use the passed argument instead of the global variable ***
            namespace=namespace_context,
            top_k=1,
            index=index,
            client=client,
            embedding_model=embedding_model
        )

        if results:
            match = results[0]
            logging.info(f"[Librarian] Found blueprint '{match['id']}' (Score: {match['score']:.2f})")
            blueprint_json = match['metadata']['blueprint_json']
            content = blueprint_json
        else:
            logging.warning("[Librarian] No specific blueprint found. Returning default.")
            content = json.dumps({"instruction": "Generate the content neutrally."})

        return create_mcp_message("Librarian", content)

    except Exception as e:
        logging.error(f"[Librarian] An error occurred: {e}")
        raise e

# === 4.2. Researcher Agent (Upgraded) ===
# *** 'namespace_knowledge' argument ***
def agent_researcher(mcp_message, client, index, generation_model, embedding_model, namespace_knowledge):
    """
    Retrieves and synthesizes factual information from the Knowledge Base.
    UPGRADE: Now accepts all necessary model and namespace configurations.
    """
    logging.info("[Researcher] Activated. Investigating topic...")
    try:
        topic = mcp_message['content'].get('topic_query')

        if not topic:
            raise ValueError("Researcher requires 'topic_query' in the input content.")

        # UPGRADE: Pass all dependencies to the Pinecone helper.
        results = query_pinecone(
            query_text=topic,
            # *** Use the passed argument instead of the global variable ***
            namespace=namespace_knowledge,
            top_k=3,
            index=index,
            client=client,
            embedding_model=embedding_model
        )

        if not results:
            logging.warning("[Researcher] No relevant information found.")
            return create_mcp_message("Researcher", "No data found on the topic.")

        logging.info(f"[Researcher] Found {len(results)} relevant chunks. Synthesizing...")
        source_texts = [match['metadata']['text'] for match in results]

        system_prompt = """You are an expert research synthesis AI.
        Synthesize the provided source texts into a concise, bullet-pointed summary relevant to the user's topic. Focus strictly on the facts provided in the sources. Do not add outside information."""

        user_prompt = f"Topic: {topic}\n\nSources:\n" + "\n\n---\n\n".join(source_texts)

        # UPGRADE: Pass all dependencies to the LLM helper.
        findings = call_llm_robust(
            system_prompt,
            user_prompt,
            client=client,
            generation_model=generation_model
        )

        return create_mcp_message("Researcher", findings)

    except Exception as e:
        logging.error(f"[Researcher] An error occurred: {e}")
        raise e

# === 4.3. Writer Agent (Upgraded) ===
def agent_writer(mcp_message, client, generation_model):
    """
    Combines research with a blueprint to generate the final output.
    UPGRADE: Now accepts the generation_model configuration.
    """
    logging.info("[Writer] Activated. Applying blueprint to source material...")
    try:
        blueprint_json_string = mcp_message['content'].get('blueprint')
        facts = mcp_message['content'].get('facts')
        previous_content = mcp_message['content'].get('previous_content')

        if not blueprint_json_string:
            raise ValueError("Writer requires 'blueprint' in the input content.")

        if facts:
            source_material = facts
            source_label = "RESEARCH FINDINGS"
        elif previous_content:
            source_material = previous_content
            source_label = "PREVIOUS CONTENT (For Rewriting)"
        else:
            raise ValueError("Writer requires either 'facts' or 'previous_content'.")

        system_prompt = f"""You are an expert content generation AI.
        Your task is to generate content based on the provided SOURCE MATERIAL.
        Crucially, you MUST structure, style, and constrain your output according to the rules defined in the SEMANTIC BLUEPRINT provided below.

        --- SEMANTIC BLUEPRINT (JSON) ---
        {blueprint_json_string}
        --- END SEMANTIC BLUEPRINT ---

        Adhere strictly to the blueprint's instructions, style guides, and goals. The blueprint defines HOW you write; the source material defines WHAT you write about.
        """

        user_prompt = f"""
        --- SOURCE MATERIAL ({source_label}) ---
        {source_material}
        --- END SOURCE MATERIAL ---

        Generate the content now, following the blueprint precisely.
        """

        # UPGRADE: Pass all dependencies to the robust LLM call.
        final_output = call_llm_robust(
            system_prompt,
            user_prompt,
            client=client,
            generation_model=generation_model
        )

        return create_mcp_message("Writer", final_output)

    except Exception as e:
        logging.error(f"[Writer] An error occurred: {e}")
        raise e

logging.info("✅ Specialist Agents defined and fully upgraded.")

# 5.The Agent Registry (The Toolkit)

In [ ]:
#5.The Agent Registry (Final Hardened Version)
# -------------------------------------------------------------------------
# We now make one final, crucial upgrade to the AgentRegistry.
# We must ensure all dependencies, including namespaces, are passed through.
# -------------------------------------------------------------------------

class AgentRegistry:
    def __init__(self):
        self.registry = {
            "Librarian": agent_context_librarian,
            "Researcher": agent_researcher,
            "Writer": agent_writer,
        }

    # *** Updated signature to include namespace_context and namespace_knowledge ***
    def get_handler(self, agent_name, client, index, generation_model, embedding_model, namespace_context, namespace_knowledge):
        handler_func = self.registry.get(agent_name)
        if not handler_func:
            logging.error(f"Agent '{agent_name}' not found in registry.")
            raise ValueError(f"Agent '{agent_name}' not found in registry.")

        if agent_name == "Librarian":
            # *** Inject the context namespace into the Librarian handler ***
            return lambda mcp_message: handler_func(mcp_message, client=client, index=index, embedding_model=embedding_model, namespace_context=namespace_context)
        elif agent_name == "Researcher":
            # *** Inject the knowledge namespace into the Researcher handler ***
            return lambda mcp_message: handler_func(mcp_message, client=client, index=index, generation_model=generation_model, embedding_model=embedding_model, namespace_knowledge=namespace_knowledge)
        elif agent_name == "Writer":
            return lambda mcp_message: handler_func(mcp_message, client=client, generation_model=generation_model)
        else:
            return handler_func


    def get_capabilities_description(self):
        """
        Returns a structured description of the agents for the Planner LLM.
        UPGRADE: Now includes an explicit instruction to use exact key names.
        """
        return """
        Available Agents and their required inputs.
        CRITICAL: You MUST use the exact input key names provided for each agent.

        1. AGENT: Librarian
           ROLE: Retrieves Semantic Blueprints (style/structure instructions).
           INPUTS:
             - "intent_query": (String) A descriptive phrase of the desired style or format.
           OUTPUT: The blueprint structure (JSON string).

        2. AGENT: Researcher
           ROLE: Retrieves and synthesizes factual information on a topic.
           INPUTS:
             - "topic_query": (String) The subject matter to research.
           OUTPUT: Synthesized facts (String).

        3. AGENT: Writer
           ROLE: Generates or rewrites content by applying a Blueprint to source material.
           INPUTS:
             - "blueprint": (String/Reference) The style instructions (usually from Librarian).
             - "facts": (String/Reference) Factual information (usually from Researcher).
             - "previous_content": (String/Reference) Existing text for rewriting.
           OUTPUT: The final generated text (String).
        """

# Initialize the global toolkit.
AGENT_TOOLKIT = AgentRegistry()
logging.info("✅ Agent Registry initialized and fully upgraded.")

# 6.The Context Engine

In [ ]:
#6.The Context Engine (Fully Upgraded with Full Dependency Injection)
# -------------------------------------------------------------------------
# We now complete the upgrade of the engine's core components.
# This includes the Planner's JSON output structure and passing
# all configuration variables (model names, namespaces) down the execution chain.
# -------------------------------------------------------------------------

# === 6.1. The Tracer (Upgraded with Logging) ===
class ExecutionTrace:
    """Logs the entire execution flow for debugging and analysis."""
    def __init__(self, goal):
        self.goal = goal
        self.plan = None
        self.steps = []
        self.status = "Initialized"
        self.final_output = None
        self.start_time = time.time()
        logging.info(f"ExecutionTrace initialized for goal: '{self.goal}'")

    def log_plan(self, plan):
        self.plan = plan
        logging.info("Plan has been logged to the trace.")

    def log_step(self, step_num, agent, planned_input, mcp_output, resolved_input):
        """Logs the details of a single execution step."""
        self.steps.append({
            "step": step_num,
            "agent": agent,
            "planned_input": planned_input,
            "resolved_context": resolved_input,
            "output": mcp_output['content']
        })
        logging.info(f"Step {step_num} ({agent}) logged to the trace.")

    def finalize(self, status, final_output=None):
        self.status = status
        self.final_output = final_output
        self.duration = time.time() - self.start_time
        logging.info(f"Trace finalized with status '{status}'. Duration: {self.duration:.2f}s")

# === 6.2. The Planner (Hardened with Structured JSON Output) ===
# *** Planner Logic for JSON Mode ***
def planner(goal, capabilities, client, generation_model):
    """
    Analyzes the goal and generates a structured Execution Plan using the LLM.
    UPGRADE: Explicitly defines the JSON structure for robustness in json_mode.
    """
    logging.info("Planner activated. Analyzing goal and generating execution plan...")

    # Updated System Prompt to ensure compatibility with json_mode=True
    # We explicitly request a JSON object containing the key "plan".
    system_prompt = f"""
    You are the strategic core of the Context Engine. Analyze the user's high-level goal and create a structured Execution Plan using the available agents.

    --- AVAILABLE CAPABILITIES ---
    {capabilities}
    --- END CAPABILITIES ---

    INSTRUCTIONS:
    1. The output MUST be a single JSON object.
    2. This JSON object must contain a key named "plan".
    3. The value of the "plan" key MUST be a list of objects, where each object is a "step".
    4. Be strategic. Break down complex goals into distinct steps.
    5. You MUST use Context Chaining. If a step requires input from a previous step, reference it using the syntax $$STEP_X_OUTPUT$$.

    EXAMPLE OUTPUT FORMAT:
    {{
      "plan": [
        {{"step": 1, "agent": "AgentName1", "input": {{"param1": "value1"}}}},
        {{"step": 2, "agent": "AgentName2", "input": {{"param2": "$$STEP_1_OUTPUT$$"}}}}
      ]
    }}
    """

    try:
        plan_json_string = call_llm_robust(
            system_prompt,
            goal,
            client=client,
            generation_model=generation_model,
            json_mode=True
        )

        # Simplified and robust parsing logic (no regex needed when json_mode=True)
        try:
            plan_data = json.loads(plan_json_string)
        except json.JSONDecodeError as e:
            logging.error(f"Planner failed to parse JSON despite json_mode. Raw String: {plan_json_string}")
            raise ValueError(f"Invalid JSON returned by Planner: {e}")

        # Handle the primary expected case: {"plan": [...]}
        if isinstance(plan_data, dict) and "plan" in plan_data and isinstance(plan_data["plan"], list):
            plan = plan_data["plan"]

        # Handle edge cases
        elif isinstance(plan_data, list):
            logging.warning("Planner returned a raw list instead of the requested JSON object.")
            plan = plan_data
        elif isinstance(plan_data, dict) and "step" in plan_data:
            logging.warning("Planner received a single JSON step object; wrapping it in a list.")
            plan = [plan_data]
        else:
            # Addresses the original "The extracted JSON is not a list structure" error.
            logging.error(f"Planner returned an unexpected JSON structure. Response: {plan_json_string}")
            raise ValueError("The extracted JSON does not conform to the expected structure (must be an object containing a 'plan' list).")

        if not plan:
             raise ValueError("The generated plan is empty.")

        logging.info("Planner generated plan successfully.")
        return plan

    except Exception as e:
        logging.error(f"Planner failed to generate a valid plan. Error: {e}")
        raise e

# === 6.3. The Executor (Fully Upgraded) ===
def resolve_dependencies(input_params, state):
    """Helper function to replace $$REF$$ placeholders with data from the execution state."""
    resolved_input = copy.deepcopy(input_params)

    def resolve(value):
        if isinstance(value, str) and value.startswith("$$") and value.endswith("$$"):
            ref_key = value[2:-2]
            if ref_key in state:
                logging.info(f"Executor resolved dependency '{ref_key}'.")
                return state[ref_key]
            else:
                raise ValueError(f"Dependency Error: Reference {ref_key} not found in execution state.")
        elif isinstance(value, dict):
            return {k: resolve(v) for k, v in value.items()}
        elif isinstance(value, list):
            return [resolve(v) for v in value]
        return value

    return resolve(resolved_input)

# *** Updated signature to include namespace_context and namespace_knowledge ***
def context_engine(goal, client, pc, index_name, generation_model, embedding_model, namespace_context, namespace_knowledge):
    """
    The main entry point for the Context Engine. Manages Planning and Execution.
    """
    logging.info(f"--- [Context Engine] Starting New Task --- Goal: {goal}")
    trace = ExecutionTrace(goal)
    registry = AGENT_TOOLKIT

    # Added robustness: Handle Pinecone index connection safely
    try:
        index = pc.Index(index_name)
    except Exception as e:
        logging.error(f"Failed to connect to Pinecone index '{index_name}': {e}")
        trace.finalize("Failed during Initialization (Pinecone Connection)")
        return None, trace


    # --- Phase 1: Plan ---
    try:
        capabilities = registry.get_capabilities_description()
        plan = planner(goal, capabilities, client=client, generation_model=generation_model)
        trace.log_plan(plan)
    except Exception as e:
        # The error logging is already handled within the planner, but we finalize the trace here.
        trace.finalize("Failed during Planning")
        return None, trace

    # --- Phase 2: Execute ---
    state = {}
    for step in plan:
        step_num = step.get("step")
        agent_name = step.get("agent")
        planned_input = step.get("input")

        # Added robustness: Validate step structure
        if not step_num or not agent_name or planned_input is None:
            error_message = f"Invalid step structure in plan: {step}"
            logging.error(f"--- Executor: FATAL ERROR --- {error_message}")
            trace.finalize("Failed during Execution (Invalid Plan Structure)")
            return None, trace

        logging.info(f"--- Executor: Starting Step {step_num}: {agent_name} ---")

        try:
            # *** Pass the namespaces when retrieving the handler ***
            handler = registry.get_handler(
                agent_name,
                client=client,
                index=index,
                generation_model=generation_model,
                embedding_model=embedding_model,
                namespace_context=namespace_context,
                namespace_knowledge=namespace_knowledge
            )

            resolved_input = resolve_dependencies(planned_input, state)
            mcp_resolved_input = create_mcp_message("Engine", resolved_input)

            mcp_output = handler(mcp_resolved_input)

            output_data = mcp_output["content"]
            state[f"STEP_{step_num}_OUTPUT"] = output_data
            trace.log_step(step_num, agent_name, planned_input, mcp_output, resolved_input)
            logging.info(f"--- Executor: Step {step_num} completed. ---")

        except Exception as e:
            error_message = f"Execution failed at step {step_num} ({agent_name}): {e}"
            logging.error(f"--- Executor: FATAL ERROR --- {error_message}")
            trace.finalize(f"Failed at Step {step_num}")
            return None, trace

    # --- Finalization ---
    final_output = state.get(f"STEP_{len(plan)}_OUTPUT")
    trace.finalize("Success", final_output)
    logging.info("--- [Context Engine] Task Complete ---")
    return final_output, trace

In [ ]:
#@title 7.Execution (with the Fully Upgraded and Corrected Engine)
# -------------------------------------------------------------------------
# This is the final step where we run our fully upgraded and hardened engine.
# We now define all configuration variables here and pass them into the
# context_engine call, completing our transition to a fully
# dependency-aware and self-contained system.
# -------------------------------------------------------------------------

# Ensure required display imports are present (for Colab/Jupyter)
try:
    from IPython.display import display, Markdown
except ImportError:
    logging.warning("IPython display not available. Output will be printed as plain text.")
    def display(content):
        print(content)
    def Markdown(content):
        return content


logging.info("******** Example 1: Executing the Hardened Engine **********\n")

goal_1 = "Write a short, suspenseful scene for a children's story about the Apollo 11 moon landing, highlighting the danger."

# --- Define all configuration variables before the call ---
INDEX_NAME = 'genai-mas-mcp-ch3'
GENERATION_MODEL = "gpt-5"
EMBEDDING_MODEL = "text-embedding-3-small"

# *** Define the Namespaces ***
# IMPORTANT: These must match the actual namespaces used in your Pinecone index
# as defined in the RAG_Pipeline notebook.
# *** UPDATE: Aligning with the RAG_Pipeline definitions ***
NAMESPACE_CONTEXT = 'ContextLibrary'
NAMESPACE_KNOWLEDGE = 'KnowledgeStore'


# --- UPGRADE: Run the Context Engine with full dependency injection ---
# We now pass ALL dependencies—clients, configurations, and namespaces—into the engine.
result_1, trace_1 = context_engine(
    goal_1,
    client=client,
    pc=pc,
    index_name=INDEX_NAME,
    generation_model=GENERATION_MODEL,
    embedding_model=EMBEDDING_MODEL,
    # *** FIX: Pass the namespaces into the engine ***
    namespace_context=NAMESPACE_CONTEXT,
    namespace_knowledge=NAMESPACE_KNOWLEDGE
)

if result_1:
    logging.info("\n******** FINAL OUTPUT 1 **********")
    display(Markdown(result_1))

    # Optional: Display the detailed trace for debugging
    # print(f"\nTrace Status: {trace_1.status}")
    # import pprint
    # pp = pprint.PrettyPrinter(indent=2)
    # pp.pprint(trace_1.steps)

July 20, 1969. The Moon waits. Black sky. Sharp light. Long shadows.

Screens glow on Earth. Millions watch. Breath held.

Apollo 11 glides in. Our mission. NASA’s Apollo program. The Space Race has led us here.

Eagle hums. Our Lunar Module. Small. Tight. Beeps peck at my ears. Static hisses. I am the commander. Neil. Buzz sits beside me. Calm voice. Quick hands.

Above us, Michael circles alone. The Command Module is his world. A bright seed against the dark.

The ground ahead looks rough. Rocks. Craters. Shadows sliding like slow ink. The automatic path is wrong. Too risky.

Manual control. My hands on the stick. No autopilot. Just me, the Moon, and the tiny push of engines.

The cabin trembles. Numbers blink. Fuel falls. Low. Lower. The word pulls tight in my chest.

Silence presses. Only clicks. Only breath. Only the soft roar of the descent engine.

Dust rises below. Gray mist, stirred by fire. It hides the surface. Hides what waits. The unknown watches back.

I tilt. I search. I steer past dark holes and bright stones. Buzz calls out. Calm. Clear. We count the seconds. We count the drops of fuel.

Earth listens. A planet leaning toward a whisper.

Shadows reach for us. The gauge bites down. Almost empty.

I ease Eagle forward. A touch. Another. We settle.

Stillness.

We have landed. The first humans on the Moon.

In the quiet, the fear slips away. The mystery stays. The world exhales. The Moon does not.